In [1]:
import pandas as pd
import numpy as np
import datetime
import os
from haversine import haversine
import json

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

print(datetime.datetime.now())
os.getcwd()

2020-03-06 13:58:05.439195


'/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/Faith_SOTF_zips_20200306'

In [3]:
# Email from Faith
# Q1 SOTF Zip Codes
# 2020-03-06 10:08 a.m.
# Monday, March 2, 2020 at 6:02 PM

store_list=[5416,5414,5412,5413,5405,4710]

TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)

TA_zips['location_id']=TA_zips['location_id'].astype(int)
TA_zips[TA_zips['location_id'].isin(store_list)]

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA


In [4]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt",
                           dtype=str,sep="|")
df_store_list['location_id']=df_store_list['location_id'].astype(int)
df_store_list=df_store_list[df_store_list['location_id'].isin(store_list)]
df_store_list['latitude_meas']=df_store_list['latitude_meas'].astype(float)
df_store_list['longitude_meas']=df_store_list['longitude_meas'].astype(float)

df_store_list

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet
1404,5414,LEE'S CROSSING SHOPPING CENTER,2020-02-26,NaN,"4125 HIGHWAY 20, STE A-2",NaN,BUFORD,GA,30518-3900,-84.00135,34.093802,00000000000000.0000,00000000000000.0000


In [5]:
# All new
# BL Store List Report from Dom 2020 0131.xlsx
dict_store_Google=df_store_list.set_index("location_id").to_dict()['latitude_meas']

# 5414
dict_store_Google.update({5414:{"Address":"4125 HIGHWAY 20, STE A-2, BUFORD, GA 30518","Coor":[34.093802,-84.00135]}})

# 5412 2120 EAST DOROTHY LANE	KETTERING TOWN CENTER	KETTERING	OH	45420
dict_store_Google.update({5412:{"Address":"2120 E Dorothy Ln, Kettering, OH 45420","Coor":[39.7016111,-84.1264227]}})

# 5413 340 SEVILLE ST	KOHLS SC	FLORENCE	AL	35634-1511
dict_store_Google.update({5413:{"Address":"340 Seville St, Florence, AL 35634","Coor":[34.8382439,-87.6269528]}})

# 5405 1300 US HIGHWAY 127	FRANKLIN SQUARE & FRANKLIN WEST SHOPPING CENTER	FRANKFORT	KY	40601-4395
dict_store_Google.update({5405:{"Address":"1300 US-127, Frankfort, KY 40601","Coor":[38.1668777,-84.9008879]}})

# 4710 255 WEDDINGTON BRANCH ROAD	MAGIC MART SC	PIKEVILLE	KY	41501-3205
dict_store_Google.update({4710:{"Address":"255 Weddington Branch Rd, Pikeville, KY 41501","Coor":[37.525769,-82.5685473]}})

# 5416 340 REMOUNT ROAD	GATEWAY PLAZA	FRONT ROYAL	VA	22630-2187
dict_store_Google.update({5416:{"Address":"340 Remount Rd, Front Royal, VA 22630","Coor":[38.9075499,-78.1841948]}})


dict_store_Google

{5414: {'Address': '4125 HIGHWAY 20, STE A-2, BUFORD, GA 30518',
  'Coor': [34.093802, -84.00135]},
 5412: {'Address': '2120 E Dorothy Ln, Kettering, OH 45420',
  'Coor': [39.7016111, -84.1264227]},
 5413: {'Address': '340 Seville St, Florence, AL 35634',
  'Coor': [34.8382439, -87.6269528]},
 5405: {'Address': '1300 US-127, Frankfort, KY 40601',
  'Coor': [38.1668777, -84.9008879]},
 4710: {'Address': '255 Weddington Branch Rd, Pikeville, KY 41501',
  'Coor': [37.525769, -82.5685473]},
 5416: {'Address': '340 Remount Rd, Front Royal, VA 22630',
  'Coor': [38.9075499, -78.1841948]}}

In [6]:
df_output=pd.DataFrame(dict_store_Google).T.reset_index()
df_output=df_output.rename(columns={"index":"location_id"})
df_output

,location_id,Address,Coor
0,5414,"4125 HIGHWAY 20, STE A-2, BUFORD, GA 30518","[34.093802, -84.00135]"
1,5412,"2120 E Dorothy Ln, Kettering, OH 45420","[39.7016111, -84.1264227]"
2,5413,"340 Seville St, Florence, AL 35634","[34.8382439, -87.6269528]"
3,5405,"1300 US-127, Frankfort, KY 40601","[38.1668777, -84.9008879]"
4,4710,"255 Weddington Branch Rd, Pikeville, KY 41501","[37.525769, -82.5685473]"
5,5416,"340 Remount Rd, Front Royal, VA 22630","[38.9075499, -78.1841948]"


In [7]:
df_store_zip=pd.DataFrame()
for ind, row in df_output.iterrows():
    location_id=row['location_id']
    location_add=row['Address']
    location_coor=row['Coor']
    list_store_zip=[]
    for zip_cd, v in zip_centers.items():
        dist=haversine(location_coor,v,unit="mi")
        if dist<=10:
            list_store_zip.append(zip_cd)
    df=pd.DataFrame({"zip_cd":list_store_zip})
    df['location_id']=location_id
    df['zip_type']="zip_10"
    df_store_zip=df_store_zip.append(df)
df_output=pd.merge(df_output,df_store_zip,on="location_id",how="left")
df_output

,location_id,Address,Coor,zip_cd,zip_type
0,5414,"4125 HIGHWAY 20, STE A-2, BUFORD, GA 30518","[34.093802, -84.00135]",30024,zip_10
1,5414,"4125 HIGHWAY 20, STE A-2, BUFORD, GA 30518","[34.093802, -84.00135]",30019,zip_10
2,5414,"4125 HIGHWAY 20, STE A-2, BUFORD, GA 30518","[34.093802, -84.00135]",30098,zip_10
3,5414,"4125 HIGHWAY 20, STE A-2, BUFORD, GA 30518","[34.093802, -84.00135]",30097,zip_10
4,5414,"4125 HIGHWAY 20, STE A-2, BUFORD, GA 30518","[34.093802, -84.00135]",30542,zip_10
...,...,...,...,...,...
88,5416,"340 Remount Rd, Front Royal, VA 22630","[38.9075499, -78.1841948]",22640,zip_10
89,5416,"340 Remount Rd, Front Royal, VA 22630","[38.9075499, -78.1841948]",22645,zip_10
90,5416,"340 Remount Rd, Front Royal, VA 22630","[38.9075499, -78.1841948]",22610,zip_10
91,5416,"340 Remount Rd, Front Royal, VA 22630","[38.9075499, -78.1841948]",22643,zip_10


In [9]:
df_output['zip_cd'].nunique(),df_output['location_id'].nunique()

(93, 6)

In [10]:
df_output_1=df_output[df_output['location_id']!=5416]
df_output_5416=df_output[df_output['location_id']==5416]

In [12]:
writer=pd.ExcelWriter("./BL_zips_for_5_stores_and_5416_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_1.to_excel(writer,"5_store_zips",index=False)
df_output_5416.to_excel(writer,"zips_5416",index=False)
writer.save()